# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd

In [2]:
con=sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [5]:
# Your code here
pd.read_sql("""
SELECT e.lastName AS 'Last', e.firstName AS 'First', o.city
FROM employees AS e
INNER JOIN offices AS o
    USING(officeCode)
WHERE o.city=='Boston'
ORDER BY 'Last'
""",con)

,Last,First,city
0,Firrelli,Julie,Boston
1,Patterson,Steve,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [6]:
# Your code here
pd.read_sql("""
SELECT o.officeCode AS 'Office Code',
       o.city,
       COUNT(e.employeeNumber) AS 'Total Employees'
FROM employees AS e
INNER JOIN offices AS o
    USING(officeCode)
GROUP BY o.officeCode
ORDER BY o.city
""",con)

,Office Code,city,Total Employees
0,2,Boston,2
1,7,London,2
2,3,NYC,2
3,4,Paris,5
4,1,San Francisco,6
5,6,Sydney,4
6,5,Tokyo,2


## Write 3 questions of your own and answer them

In [ ]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

In [11]:
"""
How many products were sold from each product line?
"""

# Your code here
pd.read_sql("""
SELECT p.productLine, SUM(o.quantityOrdered) AS 'Total Ordered'
FROM products AS p
INNER JOIN orderDetails AS o
    USING(productCode)
GROUP BY p.productLine
ORDER BY SUM(o.quantityOrdered) DESC
""",con)

,productLine,Total Ordered
0,Classic Cars,35582
1,Vintage Cars,22933
2,Motorcycles,12778
3,Planes,11872
4,Trucks and Buses,11001
5,Ships,8532
6,Trains,2818


In [12]:
"""
How many customers does each employee work with?
"""

# Your code here
pd.read_sql("""
SELECT e.lastName AS 'Emp Last Name',
       e.firstName AS 'Emp First Name',
       COUNT(c.customerNumber) AS 'Total Customers'
FROM employees AS e
INNER JOIN customers AS c
    ON c.salesRepEmployeeNumber=e.employeeNumber
GROUP BY e.employeeNumber
ORDER BY e.lastName
""",con)

,Emp Last Name,Emp First Name,Total Customers
0,Bondur,Loui,6
1,Bott,Larry,8
2,Castillo,Pamela,10
3,Firrelli,Julie,6
4,Fixter,Andy,5
5,Gerard,Martin,6
6,Hernandez,Gerard,7
7,Jennings,Leslie,6
8,Jones,Barry,9
9,Marsh,Peter,5


In [14]:
"""
Which customers have placed the most orders?
"""

# Your code here
pd.read_sql("""
SELECT c.customerName, COUNT(o.orderNumber) AS 'Orders'
FROM customers AS c
INNER JOIN orders AS o
    USING(customerNumber)
GROUP BY o.customerNumber
ORDER BY COUNT(o.orderNumber) DESC
""",con)

,customerName,Orders
0,Euro+ Shopping Channel,26
1,Mini Gifts Distributors Ltd.,17
2,Reims Collectables,5
3,"Down Under Souveniers, Inc",5
4,"Dragon Souveniers, Ltd.",5
...,...,...
93,"Clover Collections, Co.",2
94,Osaka Souveniers Co.,2
95,Cambridge Collectables Co.,2
96,Daedalus Designs Imports,2


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [17]:
# Your code here
pd.read_sql("""
WITH order_pr AS (
SELECT p.productName, od.productCode, od.orderNumber
FROM orderDetails od
INNER JOIN products p
    USING(productCode)
)

SELECT e.lastName, e.firstName, pr.productName
FROM employees e
INNER JOIN (
            SELECT c.salesRepEmployeeNumber, orders.customerNumber, orders.productName
            FROM customers c
            INNER JOIN (
                        SELECT op.productName, op.orderNumber, od.customerNumber
                        FROM orders od
                        INNER JOIN order_pr AS op
                            USING(orderNumber)) AS orders
                USING(customerNumber)) AS pr
    ON e.employeeNumber=pr.salesRepEmployeeNumber
ORDER BY e.lastName
""",con)

,lastName,firstName,productName
0,Bondur,Loui,1952 Alpine Renault 1300
1,Bondur,Loui,1962 LanciaA Delta 16V
2,Bondur,Loui,1958 Setra Bus
3,Bondur,Loui,1940 Ford Pickup Truck
4,Bondur,Loui,1926 Ford Fire Engine
...,...,...,...
2991,Vanauf,George,1956 Porsche 356A Coupe
2992,Vanauf,George,1961 Chevrolet Impala
2993,Vanauf,George,1982 Ducati 996 R
2994,Vanauf,George,1974 Ducati 350 Mk3 Desmo


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [18]:
# Your code here
pd.read_sql("""
SELECT e.lastName, e.firstName, SUM(od.quantityOrdered) AS "Products Ordered"
FROM employees e
INNER JOIN customers c
    ON e.employeeNumber=c.salesRepEmployeeNumber
JOIN orders o
    USING(customerNumber)
JOIN orderDetails od
    USING(orderNumber)
GROUP BY e.employeeNumber
ORDER BY e.lastName
""",con)

,lastName,firstName,Products Ordered
0,Bondur,Loui,6186
1,Bott,Larry,8205
2,Castillo,Pamela,9290
3,Firrelli,Julie,4227
4,Fixter,Andy,6246
5,Gerard,Martin,4180
6,Hernandez,Gerard,14231
7,Jennings,Leslie,11854
8,Jones,Barry,7486
9,Marsh,Peter,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [19]:
# Your code here
pd.read_sql("""
SELECT e.lastName, e.firstName, COUNT(DISTINCT od.productCode) AS "Diff Products Sold"
FROM employees e
JOIN customers c
    ON e.employeeNumber=c.salesRepEmployeeNumber
JOIN orders o
    USING(customerNumber)
JOIN orderDetails od
    USING(orderNumber)
GROUP BY e.employeeNumber
ORDER BY e.lastName
""",con)

,lastName,firstName,Diff Products Sold
0,Bondur,Loui,101
1,Bott,Larry,97
2,Castillo,Pamela,100
3,Firrelli,Julie,80
4,Fixter,Andy,82
5,Gerard,Martin,78
6,Hernandez,Gerard,109
7,Jennings,Leslie,107
8,Jones,Barry,98
9,Marsh,Peter,97


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!